In [ ]:
import numpy
import pandas
from matplotlib import pyplot
# ファイルの読み込み
filename = "pmea.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
#data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
# 波数範囲選択
flag = True
xmax, xmin = 3700, 3000
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
#buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
#pyplot.figure()
#pandas.Series(buff.T.sum(), index=buff.index).plot()
#pyplot.figure()
if flag: data = buff
# 平滑化
flag = True
window = 7
x = data.columns
#print("window size =", x[window - 1] - x[0])
y = data.values[-1]
from scipy.signal import savgol_filter
s = savgol_filter(y, window, 2, 0)
#pyplot.axes().set_xlim([data.columns.max(), data.columns.min()])
#pyplot.scatter(x, y, s=3, c="blue")
#pyplot.plot(x, s, c="red")
#pyplot.show()
#pyplot.axes().set_xlim([data.columns.max(), data.columns.min()])
#pyplot.scatter(x, y-s, s=3, c="blue")
#pyplot.show()
if flag:
    data = pandas.DataFrame(savgol_filter(data.values, window, 2, 0), index=data.index, columns=data.columns)
    #data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
# data < 0 を data = 0 に置換
flag = True
buff = data
buff.mask(buff < 0, 0, inplace=True)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff
# 二乗して平方根
flag = False
buff = data
buff = (buff**2)**0.5
#buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# ICA
i = 3
from sklearn.decomposition import FastICA
model = FastICA(n_components=i).fit(data.values)
loading = pandas.DataFrame(model.components_, columns=data.columns)
loading[:i].T.plot()
score = pandas.DataFrame(model.transform(data.values), index=data.index)
score.iloc[:,:i].plot()

In [ ]:
# 次元削減による初期値の準備
n = 3  # 成分数
i = 4  # 1:SVD, 2:PCA, 3: ICA, 4:NMF
from sklearn import decomposition
if i == 1: model = decomposition.TruncatedSVD(n_components=n).fit(data.values)
if i == 2: model = decomposition.PCA(n_components=n).fit(data.values)
if i == 3: model = decomposition.FastICA(n_components=n).fit(data.values)
if i == 4: model = decomposition.NMF(n_components=n, init='random',max_iter=1000).fit(data.values)
loading = pandas.DataFrame(model.components_, columns=data.columns)
loading.T.plot(xlim=[data.columns.max(), data.columns.min()])
score = pandas.DataFrame(model.fit_transform(data.values), index=data.index)
score.plot()
init_spec = loading
init_conc = score

In [ ]:
# 差スペクトルによる初期値の準備
y1 = data.values[7] - data.values[6]
y2 = data.values[12] - data.values[9]
y3 = data.values[-1] - data.values[-100]
init_spec = pandas.DataFrame([y1, y2, y3], columns=data.columns)
init_spec=(init_spec.T/init_spec.T.sum()).T
init_spec.T.plot(xlim=[data.columns.max(), data.columns.min()])
#init_spec.T.to_csv(filename[:len(filename)-4]+'_init_spec.csv')

In [ ]:
# 初期スペクトルの読み込み
filename = "pmea_init_spec.csv"
init_spec = pandas.read_csv(filename, header=0, index_col=0).T
init_spec.T.plot(xlim=[data.columns.max(), data.columns.min()])

In [ ]:
# MCR
from pymcr.mcr import McrAR
i = 2  # 1:OLS, 2:NNLS, 3:mix
if i == 1: mcr = McrAR(c_regr="OLS", st_regr="OLS")
if i == 2: mcr = McrAR(c_regr="NNLS", st_regr="NNLS")
if i == 3: mcr = McrAR(c_regr="OLS", st_regr="NNLS")

i = 1  # 1:spec, 2:conc, 3:random
if i == 1: mcr.fit(data.values, ST=init_spec)
if i == 2: mcr.fit(data.values, C=init_conc)
if i == 3: mcr.fit(data.values, C=numpy.random.rand(len(data.index), n))

ST_opt = pandas.DataFrame(mcr.ST_opt_.T, index=data.columns.T)
ST_opt.plot(xlim=[data.columns.max(), data.columns.min()])

C_opt = pandas.DataFrame(mcr.C_opt_, index=data.index)
C_opt.plot()

In [ ]:
# 結果の保存
flag = False
if flag:
    ST_opt.to_csv(filename[: len(filename) - 4] + "_opt_spec.csv")
    C_opt.to_csv(filename[: len(filename) - 4] + "_opt_conc.csv")
    data.T.to_csv(filename[: len(filename) - 4] + "_spec.csv")